Connect GPU

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Importing Libraries

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration

Model path and Image path

In [ ]:
model_path = "/storage/home/sriramk/BTP_sriramk/pixtral/models--mistral-community--pixtral-12b/snapshots/c2756cbbb9422eba9f6c5c439a214b0392dfc998" #path to base model
image_path = "/storage/home/sriramk/BTP_sriramk/images/complex_network.png"

Load image, base model and processor


In [ ]:
processor = AutoProcessor.from_pretrained(model_path)

model = LlavaForConditionalGeneration.from_pretrained(model_path)

image = Image.open(image_path).convert("RGB")

Message Prompt

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text",
             "text": "Analyze the biological network image and extract nodes, edges, reactions, ODEs, and Jacobian. Output strictly in JSON format."},
            {"type": "image", "image": image},
        ]
    }
]

Apply chat template

In [6]:
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

generate_ids = model.generate(**inputs, max_new_tokens=500)
output = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Model Generated Output

In [7]:
print(output)

Analyze the biological network image and extract nodes, edges, reactions, ODEs, and Jacobian. Output strictly in JSON format.```json
{
  "nodes": [
    {"id": "A", "type": "species"},
    {"id": "φ", "type": "species"},
    {"id": "ϕ", "type": "species"}
  ],
  "edges": [
    {"from": "A", "to": "φ", "rate": "k1"},
    {"from": "A", "to": "ϕ", "rate": "k3"},
    {"from": "ϕ", "to": "A", "rate": "k4"},
    {"from": "A", "to": "A", "rate": "k2"}
  ],
  "reactions": [
    {"reactants": ["A"], "products": ["A", "φ"], "rate": "k1"},
    {"reactants": ["A"], "products": ["A", "ϕ"], "rate": "k3"},
    {"reactants": ["ϕ"], "products": ["A"], "rate": "k4"},
    {"reactants": ["A"], "products": ["A"], "rate": "k2"}
  ],
  "ODEs": [
    {"variable": "A", "equation": "-k1 * A - k3 * A + k4 * ϕ + k2 * A"},
    {"variable": "φ", "equation": "k1 * A"},
    {"variable": "ϕ", "equation": "k3 * A - k4 * ϕ"}
  ],
  "Jacobian": {
    "dA_dA": -k1 - k3 + k2,
    "dA_φ": 0,
    "dA_ϕ": 0,
    "dφ_dA": k1,
 